## 1. Initialize constants

In [ ]:
#@title Enter client_id and client_secret
CLIENT_ID = "" #@param {type:"string"}
CLIENT_SECRET = "" #@param {type:"string"}

In [ ]:
TOKEN_URL = "https://api.fitbit.com/oauth2/token"
API_BASE = "https://api.fitbit.com"
AUTH_URI = "https://www.fitbit.com/oauth2/authorize"

# taken from -> https://dev.fitbit.com/build/reference/web-api/explore/#/
VALID_SCOPES = {
    "activity",
    "heartrate",
    "location",
    "nutrition",
    "oxygen_saturation",
    "profile",
    "respiratory_rate",
    "settings",
    "sleep",
    "social",
    "temperature",
    "weight",
}

# taken from -> https://dev.fitbit.com/build/reference/web-api/explore/#/
ENDPOINTS = {
    # User & Account
    "profile": "/1/user/-/profile.json",
    "settings": "/1/user/-/settings.json",
    # Devices & Alarms
    "devices": "/1/user/-/devices.json",
    "alarms": "/1/user/-/devices/tracker/alarm.json",
    # Friends & Subscriptions
    "friends": "/1/user/-/friends.json",
    "subscriptions": "/1/user/-/apiSubscriptions.json",
    # Activity
    "activities_summary": "/1/user/-/activities/date/{date}.json",
    "activities_logs": "/1/user/-/activities/list.json",
    "activities_goals": "/1/user/-/activities/goals/daily.json",
    "activities_lifetime": "/1/user/-/activities/lifetime.json",
    "activities_timeseries": "/1/user/-/activities/{resource}/date/{start}/{end}.json",
    "activities_intraday_1min": "/1/user/-/activities/{resource}/date/{date}/1min.json",
    "activities_intraday_15min": "/1/user/-/activities/{resource}/date/{date}/15min.json",
    # Active Zone Minutes (AZM)
    "azm_summary": "/1/user/-/activities/active-zone-minutes/date/{date}.json",
    "azm_interval": "/1/user/-/activities/active-zone-minutes/date/{date}/{interval}.json",
    "azm_intraday": "/1/user/-/activities/active-zone-minutes/date/{date}/1min.json",
    # Heart Rate
    "heartrate_summary": "/1/user/-/activities/heart/date/{date}/1d.json",
    "heartrate_timeseries": "/1/user/-/activities/heart/date/{start}/{end}.json",
    "heartrate_intraday_range": "/1/user/-/activities/heart/date/{start}/{end}/{detail_level}.json",
    "heartrate_intraday_1min": "/1/user/-/activities/heart/date/{date}/1d/1min.json",
    "heartrate_intraday_1sec": "/1/user/-/activities/heart/date/{date}/1d/1sec.json",
    # Sleep (v1.2) – preferred
    "sleep_log": "/1.2/user/-/sleep/date/{date}.json",
    "sleep_stages": "/1.2/user/-/sleep/stages/date/{date}.json",
    "sleep_timeseries": "/1.2/user/-/sleep/date/{base-date}/{end-date}.json",
    "sleep_logs_list": "/1.2/user/-/sleep/list.json",
    "sleep_log_delete": "/1.2/user/-/sleep/{logId}.json",
    # Body & Weight
    "weight_logs": "/1/user/-/body/log/weight/date/{date}.json",
    "fat_logs": "/1/user/-/body/log/fat/date/{date}.json",
    "body_goals": "/1/user/-/body/goals.json",
    "body_timeseries": "/1/user/-/body/{resource}/date/{start}/{end}.json",
    # Breathing Rate
    "breathing_summary": "/1.2/user/-/breathing-rate/date/{date}.json",
    "breathing_intraday": "/1.2/user/-/breathing-rate/date/{date}/1min.json",
    # VO2 Max (Cardio Fitness)
    "vo2_max_summary": "/1/user/-/cardio/vo2/max/date/{date}.json",
    "vo2_max_interval": "/1/user/-/cardio/vo2/max/date/{date}/{interval}.json",
    # ECG
    "ecg_logs": "/1/user/-/ecg/list.json",
    "ecg_record": "/1/user/-/ecg/{logId}.json",
    # Food & Water
    "food_logs": "/1/user/-/foods/log/date/{date}.json",
    "water_logs": "/1/user/-/foods/log/water/date/{date}.json",
    "frequent_foods": "/1/user/-/foods/log/frequent.json",
    "recent_foods": "/1/user/-/foods/log/recent.json",
    "favorite_foods": "/1/user/-/foods/log/favorite.json",
    "food_search": "/1/food/search.json",
    "food_locales": "/1/food/locales.json",
    "food_goals": "/1/user/-/foods/log/goal.json",
    "food_timeseries": "/1/user/-/foods/log/{resource}/date/{start}/{end}.json",
}


## 2. Initialize fitbit client class

In [ ]:
import requests
import pandas as pd
import urllib.parse
from requests.auth import HTTPBasicAuth
from dataclasses import dataclass


@dataclass(frozen=True)
class FitbitConfig:
    token_url: str
    api_base: str
    auth_uri: str
    valid_scopes: str
    endpoints: dict


class FitbitClient:
    def __init__(
        self,
        client_id: str,
        client_secret: str,
        access_token: str,
        refresh_token: str,
        config: FitbitConfig,
    ):
        self.client_id = client_id
        self.client_secret = client_secret
        self.access_token = access_token
        self.refresh_token = refresh_token
        self.config = config

    def _refresh_token(self):
        """Use the refresh_token to fetch a new access & refresh token pair."""
        resp = requests.post(
            self.config.token_url,
            data={
                "grant_type": "refresh_token",
                "refresh_token": self.refresh_token,
            },
            auth=HTTPBasicAuth(self.client_id, self.client_secret),
        )
        resp.raise_for_status()
        toks = resp.json()
        self.access_token = toks["access_token"]
        self.refresh_token = toks["refresh_token"]
        return self.access_token

    def _get(self, endpoint: str, params: dict = None) -> dict:
        """Low-level GET with auto-refresh on 401."""
        headers = {"Authorization": f"Bearer {self.access_token}"}
        url = f"{self.config.api_base}{endpoint}"
        resp = requests.get(url, headers=headers, params=params)
        if resp.status_code == 401:
            self._refresh_token()
            headers["Authorization"] = f"Bearer {self.access_token}"
            resp = requests.get(url, headers=headers, params=params)
        resp.raise_for_status()
        return resp.json()
    
    def exchange_authorization_code(self, code: str, redirect_uri: str) -> None:
        """
        Exchange an OAuth2 authorization code for initial access and refresh tokens.
        Updates the client’s access_token and refresh_token.
        """
        resp = requests.post(
            self.config.token_url,
            data={
                'grant_type': 'authorization_code',
                'code': code,
                'redirect_uri': redirect_uri,
            },
            auth=HTTPBasicAuth(self.client_id, self.client_secret)
        )
        resp.raise_for_status()
        toks = resp.json()
        self.access_token = toks['access_token']
        self.refresh_token = toks['refresh_token']

    def generate_authorize_url(
            self,
            redirect_uri: str,
            expires_in: int = None
        ) -> str:
        """
        Build the OAuth2 authorization URL using all configured endpoint keys as scopes.
        """
        # requested = [
        #     scope for scope in self.config.valid_scopes
        #     if any(key.startswith(scope) for key in self.config.endpoints)
        # ]
        params = {
            'response_type': 'code',
            'client_id': self.client_id,
            'redirect_uri': redirect_uri,
            'scope': ' '.join(self.config.valid_scopes)  # ' '.join(requested)
        }
        if expires_in is not None:
            params['expires_in'] = str(expires_in)
        return f"{self.config.auth_uri}?{urllib.parse.urlencode(params)}"

    def fetch_profile(self) -> dict:
        """Fetch the user's profile (contains 'memberSince')."""
        return self._get(self.config.endpoints["profile"])["user"]

    def fetch_devices(self) -> pd.DataFrame:
        """Fetch all paired devices."""
        data = self._get(self.config.endpoints["devices"])
        return pd.json_normalize(data)

    def fetch_endpoint(
        self,
        name: str,
        **kwargs
    ) -> any:
        """
        Fetch data for a single endpoint by name and return raw API response.
        Pass any template parameters (e.g., date, start, end, resource, interval, logId) via kwargs.
        """
        # Retrieve endpoint template
        tmpl = self.config.endpoints.get(name)
        if not tmpl:
            raise ValueError(f"Unknown endpoint: {name}")

        # Build URL path by injecting all provided kwargs into template
        try:
            path = tmpl.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing parameter for endpoint '{name}': {e}")

        # Perform the GET and return raw JSON
        return self._get(path)


## 3. Prepare for data extraction

In [ ]:
# Initialize Client
client = FitbitClient(
    client_id = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    access_token = "",
    refresh_token = "",
    config = FitbitConfig(
        token_url=TOKEN_URL,
        api_base=API_BASE,
        auth_uri=AUTH_URI,
        valid_scopes=VALID_SCOPES,
        endpoints=ENDPOINTS
    )
)

auth_url = client.generate_authorize_url(
    redirect_uri='http://127.0.0.1:8080/',
    expires_in=604800
)
print(auth_url)

In [ ]:
# from -> http://127.0.0.1:8080/?code=<access_token>#_=_

client.exchange_authorization_code(
    code='2d1bef9a29a564aa5c8053dbc23caf8ccb23dab7',  # put <access_token> here
    redirect_uri='http://127.0.0.1:8080/'
)

In [ ]:
client.access_token, client.refresh_token

### Overwrite `access_token` and `refresh_token` with participant's tokens

In [ ]:
# print current access_token / refresh_token
client.access_token, client.refresh_token

In [ ]:
#@title Enter participant_access_token and participant_refresh_token
PARTICIPANT_ACCESS_TOKEN = "" #@param {type:"string"}
PARTICIPANT_REFRESH_TOKEN = "" #@param {type:"string"}

In [ ]:
# overwrite
client.access_token = PARTICIPANT_ACCESS_TOKEN
client.refresh_token = PARTICIPANT_REFRESH_TOKEN
client.access_token, client.refresh_token

### Extract data

In [ ]:
NAME = "heartrate_timeseries"  # key from ENDPOINTS dict
DATE = "2025-05-31"  # date of extraction (for certain endpoints)
START_DATE = "2025-05-01"  # start_date of extraction (for certain endpoints)
END_DATE = "2025-05-31"  # end_date of extraction (for certain endpoints)
DETAIL_LEVEL = "1min"  # granularity

In [ ]:
data = client.fetch_endpoint(
    name = NAME,
    date = DATE,
    start = START_DATE,
    end = END_DATE,
    detail_level = DETAIL_LEVEL
)

pd.set_option('display.max_colwidth', None)
data